## 今天的作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

* Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。  
* Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。  
*  Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?

In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline


#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)


!pip install pingouin
import pingouin as pg
!pip install researchpy
import researchpy   
 

0.23.2


## 讀入資料

In [2]:
df_train = pd.read_csv(r'D:\ML100\Day35\Titanic_train.csv')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
#df_train['Survived_cate']=df_train['Survived']
#df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
df_train['Survived_cate'] = df_train['Survived'].astype(str)
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [4]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Age            714 non-null    float64
 1   Survived_cate  714 non-null    object 
 2   Sex            714 non-null    object 
dtypes: float64(1), object(2)
memory usage: 22.3+ KB


In [5]:
complete_data.head(5)

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male


### Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。


In [5]:
complete_data['Ageabove65'] = complete_data['Age'].apply((lambda x: 'Y' if x >=65 else 'N'))
complete_data[complete_data['Ageabove65']=='Y'].count()

Age              11
Survived_cate    11
Sex              11
Ageabove65       11
dtype: int64

In [6]:
complete_data.columns

Index(['Age', 'Survived_cate', 'Sex', 'Ageabove65'], dtype='object')

### Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
* 暗示: 觀看下面影片找答案，https://www.youtube.com/watch?v=X2d-wUt5azY

In [7]:
def mask(x):
    if x.Age >= 65 or x.Sex == 'female':
        return 'Y'
    return 'N'

complete_data['Age_above65_female'] = complete_data.apply(mask, axis=1)
complete_data.head(5)

,Age,Survived_cate,Sex,Ageabove65,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N


### Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
* 提示:    
  首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     

In [9]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 714 non-null    float64
 1   Survived_cate       714 non-null    object 
 2   Sex                 714 non-null    object 
 3   Ageabove65          714 non-null    object 
 4   Age_above65_female  714 non-null    object 
dtypes: float64(1), object(4)
memory usage: 53.5+ KB


In [16]:
#Ageabove65 和 Survived_cate皆為離散變數，運用Cramer’s V係數計算相關性，結果顯示兩者關係很薄弱
#1.用交叉列連表整理資料
contTable1 = pd.crosstab(complete_data['Ageabove65'],complete_data['Survived_cate'])
print(contTable1)

#2.計算自由度
df = min(contTable1.shape[0],contTable1.shape[1])-1
print("df=",df)

#3.運用researchpy 套件，計算出 Cramer’s V 係數 , 
crosstab, res = researchpy.crosstab(complete_data['Ageabove65'],complete_data['Survived_cate'], test='chi-square')

print(res) #res是統計檢定表
res.loc[2,'results']
print("Cramer’s V 係數=",res.loc[2,'results'])

Survived_cate    0    1
Ageabove65             
N              414  289
Y               10    1
df= 1
                Chi-square test  results
0  Pearson Chi-square ( 1.0) =    4.6035
1                    p-value =    0.0319
2               Cramer's phi =    0.0803
Cramer’s V 係數= 0.0803


In [17]:
#Age_above65_female 和 Survived_cate皆為離散變數，運用Cramer’s V係數計算相關性，結果顯示兩者關係很強
#1.用交叉列連表整理資料
contTable2 = pd.crosstab(complete_data['Age_above65_female'],complete_data['Survived_cate'])
print(contTable2)

#2.計算自由度
df = min(contTable2.shape[0], contTable2.shape[1])-1
print("df=",df)

#3.運用researchpy 套件，計算出 Cramer’s V 係數
crosstab, res = researchpy.crosstab(complete_data['Age_above65_female'],complete_data['Survived_cate'],test='chi-square')

print(res)
res.loc[2,'results']
print("Cramer’s V 係數=",res.loc[2,'results'])

Survived_cate         0    1
Age_above65_female          
N                   350   92
Y                    74  198
df= 1
                Chi-square test   results
0  Pearson Chi-square ( 1.0) =   188.6219
1                    p-value =     0.0000
2               Cramer's phi =     0.5140
Cramer’s V 係數= 0.514
